# LLM

> A module to make direct queries to OpenAI, and eventually other LLMs.

In [1]:
#| default_exp llm

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export
import os
from dotenv import load_dotenv

import openai

To run OpenAI, you'll need:
- the openai python library
- an API key, saved as an environment variable

In [4]:
#| export

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

With that done, let's try running a test query.

In [5]:
response = openai.Completion.create(model="text-davinci-003", prompt="Say this is a test", temperature=0, max_tokens=7)
response

<OpenAIObject text_completion id=cmpl-6vSBLL9w3EYWKCY0R9jgtowTcAl46 at 0x7eff7576ff40> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n\nThis is indeed a test"
    }
  ],
  "created": 1679151047,
  "id": "cmpl-6vSBLL9w3EYWKCY0R9jgtowTcAl46",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 7,
    "prompt_tokens": 5,
    "total_tokens": 12
  }
}

That worked! That said, the response is in json, so lets extract what seems most relevant, and write that into a function.

In [6]:
response['choices'][0]['text']

'\n\nThis is indeed a test'

Slightly awkward that it's still got a line break in it, but for now, we'll clean that query into a function.

In [7]:
#| export
def query_llm(prompt,model="text-davinci-003"):
    "Given a prompt, will query OpenAI and return the output"
    
    response = openai.Completion.create(model="text-davinci-003", prompt=prompt, temperature=0, max_tokens=7)

    text = response['choices'][0]['text']
    return text


In [8]:
query_llm("What is a bear?")

'\n\nA bear is a large'

That all works! I can now run `nbdev_prepare` to prepare my code to commit.

## Ingesting Data
LLMs don't know anything about how to conduct risk assessments, so we're going to make ours "read" the entirety of [Authorised Professional Practice](https://www.college.police.uk/app/using-app). For now I'm mostly copying this [OpenAI tutorial](https://platform.openai.com/docs/tutorials/web-qa-embeddings).  That's covered in the second notebook, but I've scrapped and clean the College of Policing website.  You can see that in the other module.

In [9]:
#| hide
import nbdev; nbdev.nbdev_export()